In [1]:
import data_cnn as data
from tensorflow.python.framework import ops
ops.reset_default_graph()
import numpy as np
import tensorflow as tf

In [2]:
dictLabels = data.getAllLabels('/media/data/gomez/Fully_con_NN_Data_input/split1Labels.mat')
dictData = data.getAllData('/media/data/gomez/Fully_con_NN_Data_input/Input_data1.mat')

dataTrain = dictData['train']
dataTest = dictData['test']
labelsTrain = dictLabels['train']
labelsTest = dictLabels['test']

tb_logs_path = '/home/gomez/tb_net_result'

Getting a dictionary with the labels of train and test
Retrieving labels from the file /media/data/gomez/Fully_con_NN_Data_input/split1Labels.mat from train data
Retrieving labels from the file /media/data/gomez/Fully_con_NN_Data_input/split1Labels.mat from test data
Getting a dictionary with the data of train and test
Retrieving data from the file /media/data/gomez/Fully_con_NN_Data_input/Input_data1.mat from train data
Retrieving data from the file /media/data/gomez/Fully_con_NN_Data_input/Input_data1.mat from test data
calculating max frame sequence in train and test data
Maximum number of rows found and it is: 28
from train data
from test data


In [3]:
batch_size,shpeData = dataTrain.shape
_,shpeLabl = labelsTrain.shape
print(shpeData,shpeLabl,batch_size,_)
'''batch_size,shpeData = dataTest.shape
_,shpeLabl = labelsTest.shape
print(shpeData,shpeLabl,batch_size,_)'''

#Modify the data according to the given shape above
with tf.name_scope('input'):
    x = tf.placeholder(tf.float32,[None,840])
    y_ = tf.placeholder(tf.float32,[None,12])

840 12 227 227


In [4]:
#Weight initialization

def weight_variable(shape, name):
    with tf.name_scope('weight_' + name):        
        initial = tf.truncated_normal(shape, stddev=0.1)
        w_variable = tf.Variable(initial)
        tf.scalar_summary('_weight_' + name, w_variable)
        return w_variable

def bias_variable(shape, name):
    with tf.name_scope('bias_' + name):
        initial = tf.constant(0.1, shape=shape)
        b_variable = tf.Variable(initial)
        tf.scalar_summary('_bias_' + name,b_variable)
        return b_variable

#Convolution and pooling (Martin: stride 30 for convolution, article: max_pooling of 7 with stride 7)

def conv1d(x,w):
    with tf.name_scope('Conv1d'):
        convolution = tf.nn.conv1d(x,w,stride=30, padding='SAME')    
        tf.scalar_summary('_Conv1d_',convolution)
        return convolution

def max_pool_7(x):
    with tf.name_scope('max_pool_stride7'):
        max_pool7 = tf.nn.max_pool(x,ksize=[1,1,7,1],strides=[1,1,7,1], padding='SAME') #There is just max_pool in 2d and 3d so reshape and reshape
        tf.scalar_summary('_max_pool_stride7_',max_pool7)
        return max_pool7

In [5]:
#First Layer

w_conv1 = weight_variable([30,1,32], 'conv1') #Kernel size of 30 and from 1 input to 32 neurons
b_conv1 = bias_variable([32], 'conv1') #32 biases going to different neurons

x_image = tf.reshape(x, [-1,shpeData,1]) #Image converted in 3d

h_conv1 = tf.nn.relu(conv1d(x_image, w_conv1) + b_conv1)

print(h_conv1)

'''data: [3d] -> maxpooling(data): [4d] -> reconversion: [3d]'''

# h_conv1_4d:[batch_size,1 (1d data), 28 (840/30 = after convolution 30 (filter and stride) ), 32 (the output to the new network)]

h_conv1_4d = tf.reshape(h_conv1,[batch_size,1,int(shpeData/30),32])

h_pool1 = max_pool_7(h_conv1_4d)

print(h_pool1)

#2 rshp_h_pool1:[batch_size, 4 (28/7 -> 28 from previous convolution/ 7 from maxpool filter and stride), 32 (output network)]

rshp_h_pool1 = tf.reshape(h_pool1,[batch_size,int((shpeData/30)/7),32])
print(rshp_h_pool1)

Tensor("Relu:0", shape=(?, 28, 32), dtype=float32)
Tensor("max_pool_stride7/MaxPool:0", shape=(227, 1, 4, 32), dtype=float32)
Tensor("Reshape_2:0", shape=(227, 4, 32), dtype=float32)


In [6]:
#Fully Connected layer 1

W_fc1 = weight_variable([4 * 32,128], 'fc1') #weights converted from the dimensions
b_fc1 = bias_variable([128], 'fc1')

h_pool2_flat = tf.reshape(rshp_h_pool1, [-1, 4*32])
print(h_pool2_flat)
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat,W_fc1)+b_fc1)
print(h_fc1)

Tensor("Reshape_3:0", shape=(227, 128), dtype=float32)
Tensor("Relu_1:0", shape=(227, 128), dtype=float32)


In [7]:
#Fully Connected layer 2

W_fc2 = weight_variable([128,256], 'fc2') #weights converted to 
b_fc2 = bias_variable([256], 'fc2') #Bias values going to the neurons

h_fc2 = tf.nn.relu(tf.matmul(h_fc1,W_fc2)+b_fc2) #Relu #2 -> Check if this was right -> Should be (before the h_pool2 was)
print(h_fc2)

Tensor("Relu_2:0", shape=(227, 256), dtype=float32)


In [8]:
# Dropout trick 

keep_prob = tf.placeholder("float")
h_fc2_drop = tf.nn.dropout(h_fc2,keep_prob)
print(h_fc2_drop)

# Output layer

W_fc3 = weight_variable([256,12], 'fc3')
b_fc3 = bias_variable([12], 'fc3')

y_conv = tf.nn.softmax(tf.matmul(h_fc2_drop, W_fc3) + b_fc3)
print(y_conv)

Tensor("dropout/mul_1:0", shape=(227, 256), dtype=float32)
Tensor("Softmax:0", shape=(227, 12), dtype=float32)


In [9]:
#Train and evaluate
with tf.name_scope('cross_entropy'):
    cross_entropy = -tf.reduce_sum(y_*tf.log(y_conv))
with tf.name_scope('train'):
    train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
with tf.name_scope('accuracy'):
    correct_prediction = tf.equal(tf.arg_max(y_conv,1),tf.arg_max(y_,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

tf.scalar_summary('cost', cross_entropy)
tf.scalar_summary('accuracy', accuracy)

<tf.Tensor 'ScalarSummary_1:0' shape=() dtype=string>

In [10]:
#Merging all summaries
summary_op = tf.merge_all_summaries()

In [11]:
#Starting a new session
init = tf.initialize_all_variables()
sess = tf.InteractiveSession()
sess.run(init)
writer = tf.train.SummaryWriter(tb_logs_path, graph=tf.get_default_graph())

In [12]:
for i in range(9000):    
    if i%100 == 0:
        train_accuracy = accuracy.eval(feed_dict={x:dataTrain, y_:labelsTrain, keep_prob: 0.5})
        print("step %d, training accuracy %g"%(i,train_accuracy))            
    train_step.run(feed_dict={x:dataTrain, y_:labelsTrain, keep_prob: 0.5})
    #writer.add_summary
print("test accuracy %g"%accuracy.eval(feed_dict={x:dataTest, y_:labelsTest, keep_prob: 0.5}))

step 0, training accuracy 0.0881057
step 100, training accuracy 0.295154
step 200, training accuracy 0.528634


KeyboardInterrupt: 